### Controle volledigheid resultaten

Waar wil ik inzicht in creeeren?
 1) Voor welke regio's ik resultaten heb
 2) Welke regio's mogelijk errors geven, en of ik doe nog moet en kan fixen
 
Hoe kan ik de resultaten opdelen?
 A) Grid-LUISA
 B) Grid-CORINE
 C) Main outputs
 D) Postprocessed outputs (baseline)
 E) Postprocessed outputs (cc)
 
 Waar kan het fout gaan in het model?
 I)   Postprocessing in OSM (op twee niveaus: polygonen en OSM-extracts)
 II)  Main: fetch-roads
 III) Main: damage calculations
 IV)  Postprocessing
 
 Er kan natuurlijk ook gewoon geen data zijn omdat er geen overstromings data is
 Of geen FPL; of geen AoI.
 
Welke ruimtelijke groepen?
 A) EU28; zonder Malta en Cyprus en zonder remote regions
 B) EFTA countries: zwitserland, liechtenstein, noorwegen, ijsland
 C) Entry countries: 'AL','ME','MK','RS','TR'

In [12]:
import sys
sys.path.append("..") #import folder which is one level higher

import glob as glob
import pandas as pd
import os as os

from utils_functions import load_config
from postproc_functions import NUTS_0_list, NUTS_up, NUTS_down, NUTS_3_remote

In [26]:
# Beginnen met de NUTS-3 data:
N0_EU28 = NUTS_0_list(EU28 = True, EFTA=False, CAND=False)
N3_EU28 = []
for N0 in N0_EU28:
    N3_EU28.extend(NUTS_up(N0,True))
N3_EU28

df = pd.DataFrame(index = N3_EU28)
df['EU28'] = True
df['EFTA'] = False
df['CAND'] = False

In [27]:
#append EFTA to dataframe
N0_EFTA = NUTS_0_list(EU28 = False, EFTA=True, CAND=False)
N3_EFTA = []
for N0 in N0_EFTA:
    N3_EFTA.extend(NUTS_up(N0,True))

df_temp = pd.DataFrame(index = N3_EFTA)
df_temp['EU28'] = False
df_temp['EFTA'] = True
df_temp['CAND'] = False

df = df.append(df_temp)

In [28]:
#append Candidate countries to dataframe
N0_CAND = NUTS_0_list(EU28 = False, EFTA=False, CAND=True)
N3_CAND = []
for N0 in N0_CAND:
    N3_CAND.extend(NUTS_up(N0,True))

df_temp = pd.DataFrame(index = N3_CAND)
df_temp['EU28'] = False
df_temp['EFTA'] = False
df_temp['CAND'] = True

df = df.append(df_temp)

In [29]:
#add all the outputs of the main model (vs 0.8)
main_results = load_config()['paths']['output']
os.path.exists(main_results)
main_08_VM = []
for filename in glob.glob(os.path.join(main_results,'*.pkl')):
    main_08_VM.append(filename[-9:][:5])
main_08_VM

df_temp = pd.DataFrame(index=main_08_VM)
df_temp['main_results'] = True

df2 = df.merge(df_temp,how='outer',left_index=True,right_index=True)
df2.head()

,EU28,EFTA,CAND,main_results
AL011,False,False,True,True
AL012,False,False,True,True
AL013,False,False,True,True
AL014,False,False,True,True
AL015,False,False,True,True


In [30]:
#add all the outputs of the postprocessed results (baseline)
postproc_output_baseline = os.path.join(load_config()['paths']['postproc_output'],'baseline')
postproc_baseline_09 = []
for filename in glob.glob(os.path.join(postproc_output_baseline,'*segment_nomix.pkl')):
    postproc_baseline_09.append(filename[-27:][:5])

df_temp = pd.DataFrame(index=postproc_baseline_09)
df_temp['baseline_results'] = True

df2a = df2.merge(df_temp,how='outer',left_index=True,right_index=True)
df2a.head()

,EU28,EFTA,CAND,main_results,baseline_results
AL011,False,False,True,True,True
AL012,False,False,True,True,True
AL013,False,False,True,True,True
AL014,False,False,True,True,True
AL015,False,False,True,True,True


In [31]:
#remove the remote areas
df_temp = pd.DataFrame(index = NUTS_3_remote(Overseas=True,Creta=False,Spain=True))
df_temp['Remote_areas'] = True
df3 = df2a.merge(df_temp,how='outer',left_index=True,right_index=True)

In [32]:
df3.fillna(False, inplace=True) #replace NaN with boolean False

In [33]:
df3.shape

(1522, 6)

In [43]:
df3['country'] = [x[0:2] for x in df3.index.values]

De coverage van de main_08_VM outputs:

In [34]:
df3[~df3.main_results]

,EU28,EFTA,CAND,main_results,baseline_results,Remote_areas
CY000,True,False,False,False,False,False
ES630,True,False,False,False,False,True
ES640,True,False,False,False,False,True
ES703,True,False,False,False,False,True
ES704,True,False,False,False,False,True
...,...,...,...,...,...,...
TRC22,False,False,True,False,False,False
TRC31,False,False,True,False,False,False
TRC32,False,False,True,False,False,False
TRC33,False,False,True,False,False,False


In [35]:
### Indicate the ignored countries
N0_exclude = ['MT','CY','TR','IS']

In [46]:
df3['ignored_country'] = df3['country'].isin(N0_exclude)

In [51]:
df4 = df3.loc[~(df3['ignored_country']) & ~(df3['Remote_areas'])]

In [53]:
df4[~df4['main_results']]

,EU28,EFTA,CAND,main_results,baseline_results,Remote_areas,country,ignored_country
NO053,False,True,False,False,False,False,NO,False
PL518,True,False,False,False,False,False,PL,False


Voor de regios NO053 en PL518 heb ik de oude modelresultaten (09_beta; die uit 08_VM komen) gebruikt. Daar is de HZ niet helemaal goed geinterpreteerd, maar dat scheelt maar een paar promille op het totaal.

In [59]:
df4[~df4['baseline_results']]

,EU28,EFTA,CAND,main_results,baseline_results,Remote_areas,country,ignored_country
BE256,True,False,False,True,False,False,BE,False
BE310,True,False,False,True,False,False,BE,False
BE336,True,False,False,True,False,False,BE,False
BE341,True,False,False,True,False,False,BE,False
CH053,False,True,False,True,False,False,CH,False
CH054,False,True,False,True,False,False,CH,False
CH064,False,True,False,True,False,False,CH,False
DE112,True,False,False,True,False,False,DE,False
DE131,True,False,False,True,False,False,DE,False
DE143,True,False,False,True,False,False,DE,False


In [58]:
pd.set_option("display.max_rows", 150, "display.max_columns", 20) 

# Compare with list of regions without floods

In [61]:
no_floods = ["BE256","BE336","BE341","CH053","CH054","CH064","DE112","DE131","DE143","DE218","DE21F","DE225","DE23A","DE242","DE243","DE246","DE24D","DE251","DE711","DE718","DE71B","DE804","DE912","DE916","DE944","DE945","DEA15","DEA18","DEA1A","DEA1E","DEA28","DEA2A","DEA41","DEA55","DEA5A","DEB1B","DEB24","DEB32","DEB33","DEB36","DEB37","DEB3C","DEB3D","DEB3F","DEB3H","DEC03","DEC06","DED41","DEF01","DEF04","DEG04","DEG0F","DEG0H","DK011","DK012","DK013","DK014","DK021","EL301","EL302","EL303","EL304","EL306","EL307","EL411","EL413","EL422","EL432","EL433","EL434","EL621","EL624","ES531","ES533","ES630","ES640","ES703","ES704","ES705","ES706","ES707","ES708","ES709","FI200","FRM01","IS001","ITC13","ITC32","ITC33","ITF63","ITG2A","ITG2C","MT001","MT002","NL323","NL324","NL328","NO011","PL711","PT200","PT300","RO321","UKD41","UKD46","UKD72","UKE32","UKF12","UKF21","UKG23","UKG31","UKG32","UKG33","UKG36","UKG37","UKG38","UKG39","UKH21","UKH31","UKI31","UKI61","UKI62","UKI71","UKI72","UKJ21","UKJ26","UKJ31","UKJ34","UKJ37","UKK42","UKL11","UKL15","UKL16","UKL17","UKL18","UKM64","UKM65","UKM66","UKM73","UKM75","UKM93","UKN08"]
boundary_floods = ["BE310","EL421","EL622","EL623","ITF64","ITG18","UKM78"]

In [72]:
df5 = df4.copy()

In [73]:
df5['no_floods'] = df4.index.isin(no_floods)
df5['boundary_floods'] = df4.index.isin(boundary_floods)

In [74]:
df5.loc[(~df5['no_floods']) & (~df5['boundary_floods']) & (~df5['main_results'])]

,EU28,EFTA,CAND,main_results,baseline_results,Remote_areas,country,ignored_country,no_floods,boundary_floods
NO053,False,True,False,False,False,False,NO,False,False,False
PL518,True,False,False,False,False,False,PL,False,False,False


# OKAY COOL, we are complete!!